In [1]:
from sklearn.externals import joblib

# 긍정 및 부정단어 상위 10개 출력하기

In [2]:
with open('data/movie.pkl', 'rb') as f:
    data = joblib.load(f)
locals().update(data)
f.close()

/anaconda3/lib/python3.5/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.19.1 when using version 0.20.0. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/anaconda3/lib/python3.5/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 0.19.1 when using version 0.20.0. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [3]:
tdm

<30000x1000 sparse matrix of type '<class 'numpy.float64'>'
	with 122487 stored elements in Compressed Sparse Row format>

In [4]:
from sklearn.linear_model import LogisticRegressionCV

In [5]:
from sklearn.model_selection import train_test_split

In [6]:
x_train, x_test, y_train, y_test = train_test_split(tdm, sentiment, test_size=0.2, random_state=1234)

In [7]:
model = LogisticRegressionCV()

In [8]:
model.fit(x_train, y_train)

/anaconda3/lib/python3.5/site-packages/sklearn/model_selection/_split.py:1943: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
/anaconda3/lib/python3.5/site-packages/sklearn/linear_model/logistic.py:757: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/anaconda3/lib/python3.5/site-packages/sklearn/linear_model/logistic.py:757: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/anaconda3/lib/python3.5/site-packages/sklearn/linear_model/logistic.py:757: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/anaconda3/lib/python3.5/site-packages/sklearn/linear_model/logistic.py:757: ConvergenceWarning: lbfgs failed to converge. Increase t

LogisticRegressionCV(Cs=10, class_weight=None, cv='warn', dual=False,
           fit_intercept=True, intercept_scaling=1.0, max_iter=100,
           multi_class='warn', n_jobs=None, penalty='l2',
           random_state=None, refit=True, scoring=None, solver='lbfgs',
           tol=0.0001, verbose=0)

In [9]:
model.score(x_train, y_train)

0.7542916666666667

In [10]:
model.score(x_test, y_test)

0.7283333333333334

In [11]:
model.coef_.shape

(1, 1000)

In [12]:
model.coef_[0,:10]

array([-0.96699223,  0.52531574, -0.17645564, -1.13258433,  0.66240135,
        0.22131067,  0.57946489,  0.93664739,  0.48513942,  0.24947403])

In [13]:
import pandas as pd

In [14]:
sent_df = pd.DataFrame({'단어': vectorizer.get_feature_names(), "계수" : model.coef_.flat})
sent_df.tail()

/anaconda3/lib/python3.5/site-packages/pandas/core/dtypes/dtypes.py:271: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if string == 'category':


,계수,단어
995,-0.130874,youtube
996,0.351412,yr
997,0.244320,yum
998,0.918351,yummy
999,0.466101,yup


## 긍정단어

In [15]:
pos_df = sent_df[sent_df["계수"]>0].sort_values(by='계수', ascending=False)
pos_df.head(10)
# 긍전단어 10개 출력

,계수,단어
837,2.919784,thanks
836,2.727561,thank
348,2.262597,great
941,2.199017,welcome
43,2.149099,awesome
339,1.979055,good
985,1.946974,yay
511,1.940537,love
21,1.884484,amazing
369,1.878486,happy


## 부정단어

In [16]:
neg_df = sent_df[sent_df["계수"]<0].sort_values(by='계수')
neg_df.head(10)
# 부정단어 10개

,계수,단어
707,-5.000790,sad
551,-3.204287,miss
371,-2.743394,hate
750,-2.642681,sick
780,-2.601964,sorry
950,-2.456899,wish
507,-2.417160,lost
420,-2.324808,hurts
810,-2.306172,sucks
197,-2.239247,didn


# 극성 계산하기

In [17]:
import numpy as np

In [18]:
sent_df['극성'] = np.sign(sent_df['계수'])
sent_df.set_index('단어', inplace=True)
sent_df.tail()

,계수,극성
단어,,
youtube,-0.130874,-1.0
yr,0.351412,1.0
yum,0.244320,1.0
yummy,0.918351,1.0
yup,0.466101,1.0


In [19]:
df = pd.read_csv('data/movie_test.csv')

In [20]:
df

,Test_Sentences
0,poor screen but reasonable price
1,very sad sorry for your mum
2,it's a lovely rainy day
3,really lol


In [21]:
pola = []
for sentence in df['Test_Sentences']:
    words = sentence.split()
    pola.append(sent_df.reindex(words))

## 결과 값

In [22]:
pola[0]

,계수,극성
단어,,
poor,-2.167950,-1.0
screen,-0.063689,-1.0
but,NaN,NaN
reasonable,NaN,NaN
price,NaN,NaN


In [23]:
pola[1]

,계수,극성
단어,,
very,NaN,NaN
sad,-5.000790,-1.0
sorry,-2.601964,-1.0
for,NaN,NaN
your,NaN,NaN
mum,-0.432705,-1.0


In [24]:
pola[2]

,계수,극성
단어,,
it's,NaN,NaN
a,NaN,NaN
lovely,1.137415,1.0
rainy,-0.827420,-1.0
day,0.295741,1.0


In [25]:
pola[3]

,계수,극성
단어,,
really,-0.479887,-1.0
lol,1.161912,1.0
